In [3]:
import psycopg2
import sys, nltk, re, string
from psycopg2 import Error
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from nltk.data import load
from string import punctuation
import unidecode
from es_lemmatizer import lemmatize
import spacy
from spacy_spanish_lemmatizer import SpacyCustomLemmatizer
import pandas as pd
import pandas.io.sql as sqlio
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

tweets_caso_villarejo_palabras = ['villarejo', 'ausbanc', 'corrochano', 'franciscogonzalez', 'Gonzalez', 'rato',
                                  'pineda', 'anticorrupcion', 'forense', 'casotandem', 'rick', 'fg', 'sacyr', 
                                  'bejar', 'cano', 'dineroleaks', 'cenyt', 'juez', 'fiscalia', 'bbva-villarejo']

tweets_futbol_palabras = ['futbol', 'partido', 'liga', 'clausura', 'paradas', 'goles',
                          'gol', 'estadio', 'pichichi', 'zamora', 'atajadas', 'campeon', 'copa']

tweets_palabras_positivas_es = ['bueno', 'satisfaccion', 'contento', 'agradecer', 'excelente']

tweets_palabras_negativas_es = ['hartar', 'malo', 'nefasto', 'pesimo', 'cansar', 'decepcionar', 'maldecir',
                                'corrupcion', 'odiar', 'fraude', 'reclamar', 'robo']

tweets_palabras_positivas_en = ['good', 'satisfaction', 'happy', 'appreciate', 'excellent']

tweets_palabras_negativas_en = ['sick', 'bad', 'fatal', 'grim', 'unhappy', 'damn', 'disappoint', 'hate', 
                                'corruption', 'fraud', 'steal', 'thief']

lemmatizer_en = WordNetLemmatizer()
stemmer_ingles = SnowballStemmer("english")
stemmer_español = SnowballStemmer("spanish")

def crearConexion():
    
    connection = psycopg2.connect(user = "m.soto.montesinos",
                                      password = "1234",
                                      host = "127.0.0.1",
                                      port = "5432",
                                      database = "twitterBBVA")

    return connection


def obtenerTweets():

    try:
        
        connection = crearConexion()
        query = "select status_id, text, lang from training_set_bueno"
        
        tweets = sqlio.read_sql_query(query, connection)
        
    except (Exception, psycopg2.DatabaseError) as error :
    
        print ("Error obteniendo los datos", error)
    
    finally:
        
        if (connection):
            
            connection.close()
            print("PostgreSQL connection is closed")

    return tweets


def guardar_datos_db(tweet_id, esTweet, columna):

    try:
        
        connection = crearConexion()
        cursor = connection.cursor()
        
        if columna == 'futbol':
            
            if esTweet:
                query = """
                        UPDATE training_set_bueno SET futbol = 1 where status_id = '%s';
                        """ % (tweet_id)
            else:
                query = """
                        UPDATE training_set_bueno SET futbol = 0 where status_id = '%s';
                        """ % (tweet_id)
            
        elif columna == 'caso_villarejo':
            
            if esTweet:
                query = """
                        UPDATE training_set_bueno SET caso_villarejo = 1 where status_id = '%s';
                        """ % (tweet_id)
            else:
                query = """
                        UPDATE training_set_bueno SET caso_villarejo = 0 where status_id = '%s';
                        """ % (tweet_id)
                
        elif columna == 'lista_palabras_positivas':
            
            if esTweet:
                query = """
                        UPDATE training_set_bueno SET lista_palabras_positivas = 1 where status_id = '%s';
                        """ % (tweet_id)
            else:
                query = """
                        UPDATE training_set_bueno SET lista_palabras_positivas = 0 where status_id = '%s';
                        """ % (tweet_id)
                
        elif columna == 'lista_palabras_negativas':
            
            if esTweet:
                query = """
                        UPDATE training_set_bueno SET lista_palabras_negativas = 1 where status_id = '%s';
                        """ % (tweet_id)
            else:
                query = """
                        UPDATE training_set_bueno SET lista_palabras_negativas = 0 where status_id = '%s';
                        """ % (tweet_id)
        
        cursor.execute(query)
        connection.commit()
        
    except (Exception, psycopg2.DatabaseError) as error :
    
        print ("Error obteniendo los datos", error)
    
    finally:
        
        if (connection):
            
            connection.close()
            cursor.close()
            print("PostgreSQL connection is closed")
            
            
def guardar_mayusculas_db(tweet_id, mayusculas):

    try:
        
        connection = crearConexion()
        cursor = connection.cursor()
        
        query = """
                UPDATE training_set_bueno SET mayusculas = '%f'  where status_id = '%s';
                """ % (mayusculas, tweet_id)
        
        cursor.execute(query)
        connection.commit()
        
    except (Exception, psycopg2.DatabaseError) as error :
    
        print ("Error obteniendo los datos", error)
    
    finally:
        
        if (connection):
            
            connection.close()
            cursor.close()
            print("PostgreSQL connection is closed")
            
            
def limpiar_tweet_v1(tweet):
    
    tweet   = re.sub(r'\$\w*', '', tweet)
    tweet   = re.sub(r'http?:.*$', '', tweet)
    tweet   = re.sub(r'https?:.*$', '', tweet)
    tweet   = re.sub(r'pic?.*\/\w*', '', tweet)
    tweet   = re.sub(r'[' + string.punctuation + ']+', ' ', tweet)
    
    return tweet


def limpiar_tweet_v2(tweet):
    
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'http?:.*$', '', tweet)
    tweet = re.sub(r'https?:.*$', '', tweet)
    tweet = re.sub(r'pic?.*\/\w*', '', tweet)
    tweet = re.sub('#\S+', '', tweet)
    tweet = re.sub('@\S+', '', tweet)
    tweet = tweet.replace("BBVA", "")
    tweet   = re.sub(r'[' + string.punctuation + ']+', '', tweet)
    tweet = tweet.replace(" ", "")
    
    return tweet


def limpiar_tweet_v3(tweet):
    
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'http?:.*$', '', tweet)
    tweet = re.sub(r'https?:.*$', '', tweet)
    tweet = re.sub(r'pic?.*\/\w*', '', tweet)
    tweet = re.sub('#\S+', '', tweet)
    tweet = re.sub('@\S+', '', tweet)
    tweet = tweet.replace("BBVA", "")
    tweet   = re.sub(r'[' + string.punctuation + ']+', '', tweet)
    
    return tweet


def tokenizar_tweet(tweet, lista_palabras):
    
    tweetEncontradoLista = False
    
    tokens = TweetTokenizer(strip_handles = True, reduce_len = True).tokenize(tweet)
    tokens = [w.lower() for w in tokens]
    tokens = [unidecode.unidecode(w) for w in tokens]
    
    if any(w in lista_palabras for w in tokens):
        tweetEncontradoLista = True
    
    return " ".join(tokens), tweetEncontradoLista


def obtener_mayusculas(tweet):
    
    tweet = limpiar_tweet_v2(tweet)
    
    longitud = len(tweet)
    mayusculas = sum(1 for caracter in tweet if caracter.isupper())
    porcentaje = 0
    
    if mayusculas > 0 and longitud > 0:
        porcentaje = round(mayusculas/ longitud, 2)
        
    print(tweet + " --- " + str(mayusculas) + " --- " + str(porcentaje))
    
    return porcentaje


def tweet_en_lista_palabras(idioma, positivo, tweet):
    
    lista_palabras = []
    tweet_stem = []
    tokens = TweetTokenizer(strip_handles = True, reduce_len = True).tokenize(tweet)
    tweetEncontrado = False
    
    if idioma == "en":
        
        if positivo:
            
            for w in tweets_palabras_positivas_en:
                lista_palabras.append(stemmer_ingles.stem(w))
                
            
        else:
            
            for w in tweets_palabras_negativas_en:
                
                lista_palabras.append(stemmer_ingles.stem(w))
                
        for w in tokens:
            
            tweet_stem.append(stemmer_ingles.stem(w))
        
    else:

        if positivo:
            
            for w in tweets_palabras_positivas_es:
            
                lista_palabras.append(stemmer_español.stem(w))
            
        else:
            
            for w in tweets_palabras_negativas_es:
            
                lista_palabras.append(stemmer_español.stem(w))
                
        for w in tokens:
            
            tweet_stem.append(stemmer_español.stem(w))
            
    if any(w in lista_palabras for w in tweet_stem):
        tweetEncontrado = True
            
    return lista_palabras, " ".join(tweet_stem), tweetEncontrado


def main():
    
    tweets = obtenerTweets()
    
    for index, row in tweets.iterrows():
    
        tweet_caso_villarejo = (row['text'])
        tweet_futbol = (row['text'])
        tweet_mayusculas = (row['text'])
        tweet_lista_palabras = (row['text'])
        lng_txt = (row['lang'])
        tweet_id = (row['status_id'])
        
        #tweet_caso_villarejo, esCasoVillarejo = tokenizar_tweet(limpiar_tweet_v1(tweet_caso_villarejo), tweets_caso_villarejo_palabras)
        #guardar_datos_db(tweet_id, esCasoVillarejo, "caso_villarejo")
        
        tweet_futbol, esTweetFutbol = tokenizar_tweet(limpiar_tweet_v1(tweet_futbol), tweets_futbol_palabras)
        guardar_datos_db(tweet_id, esTweetFutbol, "futbol")
        
        #mayusculas = obtener_mayusculas(tweet_mayusculas)
        #guardar_mayusculas_db(tweet_id, mayusculas)
        
        #lista_palabras, tweet_stem, tweetEncontradoPositivo = tweet_en_lista_palabras(lng_txt, True, 
        #                                                                              tweet_lista_palabras)
        #guardar_datos_db(tweet_id, tweetEncontradoPositivo, "lista_palabras_positivas")
        
        #lista_palabras, tweet_stem, tweetEncontradoNegativo = tweet_en_lista_palabras(lng_txt, False, 
        #                                                                              tweet_lista_palabras)
        #guardar_datos_db(tweet_id, tweetEncontradoNegativo, "lista_palabras_negativas")
        
        #print(str(index) + " ---- " + tweet_lista_palabras + " ---- " + tweet_stem + " ---- Positivo: " + 
        #     str(tweetEncontradoPositivo) + " ---- Negativo: " +  str(tweetEncontradoNegativo))

if __name__ == '__main__':
    main()
     





PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreS

PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreS

PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreS

PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreS

PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreS

...

PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreS

PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreS

PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreS

PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreS

PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreS